# On-Axis Field of a Solenoid Demo

This demo compares the numerically computed on-axis magnetic field of a model solenoid with the known analytical formula.

A solenoid is modeled as a series of discrete current rings. The total magnetic field is the sum of the fields from each ring. We compute the Taylor series for this field along the solenoid's axis and compare its coefficients with the coefficients from the Taylor series of the analytical formula.

In [1]:
import numpy as np
from MTFLibrary import *
from MTFLibrary.EMLibrary.biot_savart import serial_biot_savart
from MTFLibrary.EMLibrary.current_ring import current_ring
from MTFLibrary.taylor_function import MultivariateTaylorFunctionBase
import math
import pandas as pd

# --- Global MTF Settings ---
initialize_mtf_globals(max_order=6, max_dimension=4)
set_global_etol(1e-12)

# --- Define Variables for MTF ---
z = Var(3) # We only need the variable for the axis

Initializing MTF globals with: _GLOBAL_MAX_ORDER=6, _GLOBAL_MAX_DIMENSION=4
Loading/Precomputing Taylor coefficients up to order 6
Global precomputed coefficients loading/generation complete.
Size of precomputed_coefficients dictionary in memory: 464 bytes, 0.45 KB, 0.00 MB
MTF globals initialized: _GLOBAL_MAX_ORDER=6, _GLOBAL_MAX_DIMENSION=4, _INITIALIZED=True
Max coefficient count (order=6, nvars=4): 210
Precomputed coefficients loaded and ready for use.


### 1. Model the Solenoid and Compute Numerical B-Field

In [2]:
# --- Solenoid Parameters ---
solenoid_radius = 0.1
solenoid_length = 0.5
num_rings = 50 # Number of rings to approximate the solenoid
current = 1.0
turns_per_meter = num_rings / solenoid_length

# --- Field Point on the axis ---
field_point_mtf = np.array([[0, 0, z]], dtype=object)

# --- Calculate B-field by summing contributions from each ring ---
total_B_numerical = np.array([MultivariateTaylorFunctionBase.from_constant(0.0) for _ in range(3)], dtype=object)

ring_positions_z = np.linspace(-solenoid_length / 2, solenoid_length / 2, num_rings)

for z_pos in ring_positions_z:
    ring_center = np.array([0, 0, z_pos])
    ring_axis = np.array([0, 0, 1])
    num_segments_per_ring = 20

    segment_mtfs, element_lengths, direction_vectors = current_ring(
        solenoid_radius, num_segments_per_ring, ring_center, ring_axis)
    
    # Biot-Savart law returns a field that is still a function of the ring parameter 'u' (Var(4))
    B_ring_with_u = serial_biot_savart(segment_mtfs, element_lengths, direction_vectors, field_point_mtf)
    
    # Integrate over 'u' to get the total field from this one ring
    B_ring = [integrate(bfld, 4, -1, 1) for bfld in B_ring_with_u[0]]
    
    # Add this ring's contribution to the total
    total_B_numerical += B_ring

B_z_numerical = total_B_numerical[2] # We only care about the Z-component

print("Computed Bz from Solenoid Model (Taylor Series Coefficients):")
print(B_z_numerical.get_tabular_dataframe())

Computed Bz from Solenoid Model (Taylor Series Coefficients):
          Coefficient  Order     Exponents
0  1.146585093572e-04      0  (0, 0, 0, 0)
1 -3.048515550090e-04      2  (0, 0, 2, 0)
2 -5.186877015957e-03      4  (0, 0, 4, 0)
3 -5.974433014965e-02      6  (0, 0, 6, 0)


### 2. Implement the Analytical Formula

In [3]:
mu_0 = 4 * math.pi * 1e-7

# Analytical formula for the on-axis field of a finite solenoid
# B_z = (mu_0 * n * I / 2) * (cos(theta_at_end_1) - cos(theta_at_end_2))

# Define the positions of the solenoid ends
z1 = -solenoid_length / 2
z2 = solenoid_length / 2

# The arguments for the cosines are based on the displacement from the field point z
# to the ends of the solenoid.
cos_theta_1 = (z - z1) / sqrt_taylor((z - z1)**2 + solenoid_radius**2)
cos_theta_2 = (z - z2) / sqrt_taylor((z - z2)**2 + solenoid_radius**2)

B_z_analytical = (mu_0 * turns_per_meter * current / 2) * (cos_theta_1 - cos_theta_2)

print("Analytical Bz for Solenoid (Taylor Series Coefficients):")
print(B_z_analytical.get_tabular_dataframe())

Analytical Bz for Solenoid (Taylor Series Coefficients):
          Coefficient  Order     Exponents
0  1.166758220446e-04      0  (0, 0, 0, 0)
1 -3.329631069049e-04      2  (0, 0, 2, 0)
2 -5.806728776779e-03      4  (0, 0, 4, 0)
3 -6.766461594820e-02      6  (0, 0, 6, 0)


### 3. Compare the Results

In [4]:
print("--- Comparison of Taylor Series Coefficients (Bz component) ---")
df_num = B_z_numerical.get_tabular_dataframe().rename(columns={'Coefficient': 'Numerical'})
df_an = B_z_analytical.get_tabular_dataframe().rename(columns={'Coefficient': 'Analytical'})
comparison = pd.merge(df_num, df_an, on=['Order', 'Exponents'], how='outer').fillna(0)

comparison['RelativeError'] = np.abs(comparison['Numerical'] - comparison['Analytical']) / np.abs(comparison['Analytical'])
comparison['RelativeError'] = comparison['RelativeError'].replace([np.inf, -np.inf], 0).fillna(0)

print(comparison[['Exponents', 'Order', 'Numerical', 'Analytical', 'RelativeError']])

--- Comparison of Taylor Series Coefficients (Bz component) ---
      Exponents  Order           Numerical          Analytical      RelativeError
0  (0, 0, 0, 0)      0  1.146585093572e-04  1.166758220446e-04 1.728989478714e-02
1  (0, 0, 2, 0)      2 -3.048515550090e-04 -3.329631069049e-04 8.442842859441e-02
2  (0, 0, 4, 0)      4 -5.186877015957e-03 -5.806728776779e-03 1.067471522523e-01
3  (0, 0, 6, 0)      6 -5.974433014965e-02 -6.766461594820e-02 1.170521060020e-01
